### 1. 문제 정의
- 보스턴 주택 가격 데이터를 사용하여 주택 가격을 예측해보자.
- 회귀모델
- LinearRegression(), SGDRegressor()

### 2. 데이터 수집
- csv파일

In [1]:
# 필요한 라이브러리 호출
import pandas as pd

In [10]:
boston_data = pd.read_csv('./data/boston_housing.csv')
boston_data = boston_data.iloc[  : , 1 : ] # CRIM ~ MEDV 컬럼까지 추출
boston_data

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0


In [26]:
boston_data.keys()

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT', 'MEDV'],
      dtype='object')

In [37]:
# 문제데이터(X), 답 데이터(y) 분리
X = boston_data.drop('MEDV', axis= 1)
X
# 답 데이터(MEDV 컬럼)
y = boston_data['MEDV']
y




0      24.0
1      21.6
2      34.7
3      33.4
4      36.2
       ... 
501    22.4
502    20.6
503    23.9
504    22.0
505    11.9
Name: MEDV, Length: 506, dtype: float64

In [ ]:
# 문제 데이터(X) 정보 확인
X.info() # 수치형 o
X.ndim # 2차원

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    float64
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    float64
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
dtypes: float64(13)
memory usage: 51.5 KB


2

### 3. 데이터 전처리
- 결측치, 이상치 확인하는 단계
- 학습용 데이터로 전처리 생략

#### 3-1. 특성확장
- 특성(문제)들끼리 곱해서 새로운 특성을 만들어보자 --> Linear 모델의 성능을 높이기 위해서

In [42]:
# 특성(문제) 이름 확인
col = X.columns
col

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT'],
      dtype='object')

In [47]:
# 특성의 개수 확인
col.size # 13개 컬럼

13

In [ ]:
# 특성확장을 위한 반복문

# 1. 전체 컬럼을 순서대로 꺼내오는 for
for i in range(col.size) :
    # 2. 곱해지는 컬럼을 정하는 for문 한 개 필요(0 ~ 12, 1 ~ 12, 2 ~ 12, .... 12 ~ 12)
    for j in range(i, col.size) :
        X[col[i] + '*' + col[j]] = X[col[i]] * X[col[j]]
X.columns

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       ...
       'TAX*TAX', 'TAX*PTRATIO', 'TAX*B', 'TAX*LSTAT', 'PTRATIO*PTRATIO',
       'PTRATIO*B', 'PTRATIO*LSTAT', 'B*B', 'B*LSTAT', 'LSTAT*LSTAT'],
      dtype='object', length=104)